## Gal3D Example: Ellipsoid_S
This example demonstrates how to use Gal3D for three-dimensional modeling of particle distributions in simulated galaxies.

First, Gal3D requires the coordinates (`pos`) and masses (`mass`) of the particles. Here, we use AnastrisTNG to extract stellar particle data from a galaxy in the TNG50 simulation.

In [1]:
import numpy as np
from AnastrisTNG import TNGsimulation

In [2]:
path = '/home/yxi/Simulation/sims/TNG50-1/output'
snap =99
snapshot = TNGsimulation.Snapshot(path,snap)

In [3]:
ID = 516760 
sub = snapshot.load_particle(ID,order='star')
sub.physical_units()
coor_trans = sub.face_on(alignwith='star',rmax=8)

pos = sub.s['pos']
mass = sub.s['mass']

Now, let's begin the analysis with Gal3D.

In [4]:
from gal3d.analyzer import Gal3DAnalyzer
from gal3d.visualization import ModelProjector, show_image_model_residual
from gal3d.characterization import Characterizer

Next, pass `pos` and `mass` to Gal3D for analysis.

> The parameter `res_r` sets the spatial resolution. If not specified, the program will automatically adjust it based on the particle distribution.

In [5]:
gal = Gal3DAnalyzer.analyze(pos,mass,res_r=0.288)

[2025-07-29 18:47:16.180] < gal3d.analyzer > | INFO | Starting analysis...
[2025-07-29 18:47:16.188] < gal3d.particle.global_calculator > | INFO | Using a begin_r= 333.00
[2025-07-29 18:47:16.365] < gal3d.particle.global_calculator > | INFO | Iteration num= 27
[2025-07-29 18:47:16.366] < gal3d.particle.global_calculator > | INFO | After iteration, final_r= 0.03
[2025-07-29 18:47:16.380] < gal3d.particle.global_calculator > | INFO | Recentered positions by subtracting center: [-0.0008159  -0.00088915  0.00077898]
[2025-07-29 18:47:16.385] < gal3d.particle.density_estimator > | INFO | Find DensityEstimatorPlugin: DensityEstimatorKNN and load successfully
[2025-07-29 18:47:16.386] < gal3d.particle.density_estimator > | INFO | Successfully loaded density estimator plugins
[2025-07-29 18:47:16.387] < gal3d.particle.density_estimator.DensityEstimatorKNN > | INFO | cpu nums: 16
[2025-07-29 18:47:16.388] < gal3d.particle.density_estimator.DensityEstimatorKNN > | INFO | Build KDTree with option

Then, fit the three-dimensional morphology. Here, we set 300 steps; the fitting process takes about half a minute.

In [6]:
res_ellipsoid_s = gal.fit(num_step=300)

[2025-07-29 18:47:32.221] < gal3d.analyzer > | INFO | Use get_ell_structure workflow
Fitting radii: 100%|██████████| 300/300 [00:26<00:00, 11.35it/s]


You can further analyze the results using Gal3D's built-in tools. This section demonstrates how to measure the bar structure of the galaxy.

In [7]:
bar = Characterizer.get_plugin('Bar')
data = {i: res_ellipsoid_s[i] for i in ['a','eps_ab','eps_bc','ang1','ang2','ang3']}
data['pa'] = data['ang1']
bar(data).measure()

[2025-07-29 18:48:22.185] < gal3d.characterization.characterizer > | INFO | CharacterizerPlugin found: Bar and loaded successfully
[2025-07-29 18:48:22.186] < gal3d.characterization.characterizer > | INFO | Successfully loaded Characterizer plugins


{'flag': 1,
 'eps_max': np.float64(0.27202509133162683),
 'R_max': np.float64(1.258510412566927),
 'R_bar': np.float64(1.4783345469547913)}

Finally, visualize the fitted model alongside the real data.

In [ ]:
ellipsoid_s_model = ModelProjector.get_plugin('ProjectorLineIntegration')(res_ellipsoid_s)

[2025-07-29 18:48:28.293] < gal3d.visualization.model_projector > | INFO | ModelProjectorPlugin found: ProjectorLineIntegration and loaded successfully
[2025-07-29 18:48:28.294] < gal3d.visualization.model_projector > | INFO | ModelProjectorPlugin found: ProjectorSphGrid and loaded successfully
[2025-07-29 18:48:28.294] < gal3d.visualization.model_projector > | INFO | Successfully loaded model projector plugins


In [ ]:
box_lh_max =np.max(res_ellipsoid_s['a'])*1.1
zoom_lh_max = box_lh_max/4
show_image_model_residual(gal.particle,ellipsoid_s_model,
                          large_box_x_range = (-box_lh_max,box_lh_max),
                          large_box_y_range= (-box_lh_max,box_lh_max),
                          zoom_x_range=(-zoom_lh_max,zoom_lh_max),
                          zoom_y_range=(-zoom_lh_max,zoom_lh_max),
                          depth_z_range=(-box_lh_max,box_lh_max),
                          nbins_large = 500,
                          nbins_zoom = 150,
                          nlevels_large=0,
                          nlevels_zoom=22)